## Consensus Signatures

Gregory Way, modified from code written by Adeniyi Adeboye

A consensus signature can be defined as a perturbation-specific summary profile acquired by aggregating replicate level information.

### - Consensus Datasets

1. Median Aggregation
   - consensus_median (whole plate normalization)
   - consensus_median_dmso (dmso normalization).
   
2. Modified Z Score Aggregation (MODZ)
   - consensus_modz (whole plate normalization)
   - consensus_modz_dmso (dmso normalization)

The first approach weights each replicate equally.
The second approach weights replicates by average similarity to other replicates.



### The goal here:

- is to determine the median score of each MOA (Mechanism of action) based on taking the median of the correlation values between compounds of the same MOA.

We do not adjust for dose in this notebook.

In [1]:
import os
import pathlib
import pandas as pd
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
from pycytominer import feature_select
from statistics import median
import random
from scipy import stats
import pickle
from io import BytesIO
from urllib.request import urlopen
from zipfile import ZipFile

%matplotlib inline
sns.set_style("darkgrid")

In [2]:
def feature_selection(dataset_link):
    """
    Perform feature selection by dropping columns with null or 
    only zeros values, and highly correlated values from the data.
    
    params: 
    dataset_link: string of github link to the consensus dataset

    Returns:
    data: returned consensus dataframe
    
    """
    data = pd.read_csv(dataset_link, compression='gzip', error_bad_lines=False)
    cols = data.columns.tolist()
    drop_cols = [x for x in cols if ((data[x].isnull().sum()) | all(y == 0.0 for y in data[x].values))]
    data.drop(drop_cols, axis = 1, inplace = True)
    data = feature_select(
        data,
        operation=["correlation_threshold", "variance_threshold", "blocklist"],
        blocklist_file="https://raw.githubusercontent.com/broadinstitute/lincs-cell-painting/1769b32c7cef3385ccc4cea7057386e8a1bde39a/utils/consensus_blocklist.txt"
    )
    return data

In [3]:
commit = "e9737c3e4e4443eb03c2c278a145f12efe255756"

consensus_modz_link = f'https://github.com/broadinstitute/lincs-cell-painting/blob/{commit}/spherized_profiles/consensus/2016_04_01_a549_48hr_batch1_dmso_spherized_profiles_with_input_normalized_by_dmso_consensus_modz.csv.gz?raw=true'

In [4]:
data = feature_selection(consensus_modz_link)

In [5]:
data.shape

(10752, 1029)

In [6]:
data_dir = pathlib.Path("../../Profiles_level4/L1000/L1000_figshare_data")
os.listdir(data_dir) ##files in L1000 downloaded dataset

['level_4_zspc_n27837x978.gctx',
 'level_4W_zspc_n27837x978.gctx',
 'level_3_q2norm_n27837x978.gctx',
 'level_5_rank_n9482x978.gctx',
 'level_5_modz_n9482x978.gctx',
 'col_meta_level_5_n169494.txt',
 'level_5_modz_cid_n9482.grp',
 'set_size_3_level_3_REP.A_A549_only_all_compounds_and_random_well_DMSO.gmt',
 'col_meta_level_3_REP.A_A549_only_n27837.txt',
 'REP.A_A549_pert_info.txt',
 'level_5_modz_common_sigs_n8370x978.gctx',
 'col_meta_level_3_n421176.txt',
 'col_meta_level_5_REP.A_A549_only_n9482.txt']

### Mechanism of actions (MOAs) - Alignment of L1000 and Cell Painting MOAs

- Align the **L1000 pert_info meta_data** with the **Cell-painting meta_data** based on **broad id** and then further fill in some null values in cell painting MOA column with corresponding L1000 MOAs of the same broad sample id and do the same thing for the L1000 data, then take the L1000 moas as the one that will be used for further analysis (because it has the most distinct MOAs).

In [7]:
def merge_align_moa(data_dir, cp_moa_link, data):
    
    """
    This function aligns L1000 MOAs with the cell painting MOAs 
    and further fill null MOAs in one of the them (cell painting or L1000)
    with another, so far they are of the same broad sample ID.
    
    It also merge the aligned MOA metadata dataframe with the consensus data
    based on 'broad_sample_id' and outputs the dataframe with MOAs and another one
    where the broad samples has no MOAs (null moa values).
    
    params: 
    data_dir: directory that contains L1000 files
    cp_moa_link: github link to cell painting MOA metadata information .csv file
    data: consensus dataframe

    Returns:
    data_moa: merged consensus dataframe with moas
    no_moa_data: merged consensus dataframe without moas
    """
    
    df_pertinfo_cp = pd.read_csv(cp_moa_link, sep="\t")
    df_pertinfo_L1000 = pd.read_csv(os.path.join(data_dir, 'REP.A_A549_pert_info.txt'), delimiter = "\t")
    df_pertinfo_L1000.rename(columns={"pert_id": "broad_id", "pert_iname": "pert_iname_L1000", "moa": "moa_L1000"}, 
                             inplace = True)
    df_pertinfo_cp.rename(columns={"pert_iname": "pert_iname_cell_painting", "moa": "moa_cell_painting"},
                          inplace = True)
    df_pertinfo = pd.merge(df_pertinfo_L1000, df_pertinfo_cp, on=['broad_id'], how='outer')
    
    ##fill NaNs moa_L1000, pert_iname_L1000, with corresponding values in cell_painting and VICE VERSA for Cell_Painting
    df_pertinfo['moa_L1000'].fillna(value=df_pertinfo['moa_cell_painting'], inplace=True)
    df_pertinfo['pert_iname_L1000'].fillna(value=df_pertinfo['pert_iname_cell_painting'], inplace=True)
    df_pertinfo['moa_cell_painting'].fillna(value=df_pertinfo['moa_L1000'], inplace=True)
    df_pertinfo['pert_iname_cell_painting'].fillna(value=df_pertinfo['moa_L1000'], inplace=True)
    
    df_pertinfo = df_pertinfo[['broad_sample', 'broad_id', 'pert_iname_L1000', 'moa_L1000']].copy()
    df_pertinfo.rename(columns={"pert_iname_L1000": "pert_iname", "moa_L1000":"moa", "broad_sample":'Metadata_broad_sample'},
                       inplace = True)
    df_pertinfo['Metadata_broad_sample'].fillna('DMSO', inplace=True)
    data_moa = data.merge(df_pertinfo, on='Metadata_broad_sample', how = 'outer')
    no_moa_data = data_moa[data_moa['moa'].isnull()].copy().reset_index(drop = True)
    data_moa.drop(data_moa[data_moa['moa'].isnull()].index, inplace = True)
    data_moa.reset_index(drop= True, inplace = True)
    for col in ['pert_iname', 'moa']:
        data_moa[col] = data_moa[col].apply(lambda x: x.lower())
        
    return data_moa, no_moa_data

In [8]:
moa_dataset = "https://github.com/broadinstitute/lincs-cell-painting/blob/master/metadata/moa/repurposing_info_external_moa_map_resolved.tsv?raw=true"
df_all_moa, df_no_moa = merge_align_moa(data_dir, moa_dataset, data)

df_all_moa.loc[df_all_moa.Metadata_broad_sample == 'DMSO', "Metadata_dose_recode"] = 0

print(df_all_moa.shape)
df_all_moa.head()

(10674, 1032)


,Metadata_Plate_Map_Name,Metadata_cell_id,Metadata_broad_sample,Metadata_pert_well,Metadata_mmoles_per_liter,Metadata_dose_recode,Metadata_time_point,Metadata_moa,Metadata_target,Cells_AreaShape_Area,...,Nuclei_Texture_Variance_DNA_20_0,Nuclei_Texture_Variance_ER_10_0,Nuclei_Texture_Variance_ER_20_0,Nuclei_Texture_Variance_Mito_20_0,Nuclei_Texture_Variance_RNA_10_0,Nuclei_Texture_Variance_RNA_20_0,Nuclei_Texture_Variance_RNA_5_0,broad_id,pert_iname,moa
0,C-7161-01-LM6-001,A549,BRD-A25234499-001-18-3,B13,10.00000,6,48H,glucocorticoid receptor antagonist,CYP11A1|CYP19A1,-0.891810,...,-1.967960,0.296439,-0.850732,0.194639,0.503985,1.005990,0.457072,BRD-A25234499,aminoglutethimide,glucocorticoid receptor antagonist
1,C-7161-01-LM6-001,A549,BRD-A25234499-001-18-3,B14,3.33330,5,48H,glucocorticoid receptor antagonist,CYP11A1|CYP19A1,-1.029280,...,-0.042341,-1.438650,-0.148379,-0.662561,-1.220780,-0.419900,0.979208,BRD-A25234499,aminoglutethimide,glucocorticoid receptor antagonist
2,C-7161-01-LM6-001,A549,BRD-A25234499-001-18-3,B15,1.11110,4,48H,glucocorticoid receptor antagonist,CYP11A1|CYP19A1,-0.653116,...,-0.638129,-0.043856,-1.099500,-1.498490,-0.689213,-0.845814,0.408899,BRD-A25234499,aminoglutethimide,glucocorticoid receptor antagonist
3,C-7161-01-LM6-001,A549,BRD-A25234499-001-18-3,B16,0.37037,3,48H,glucocorticoid receptor antagonist,CYP11A1|CYP19A1,-0.548529,...,-0.774513,0.724588,-0.224365,-0.086897,-0.229689,-1.735200,0.457384,BRD-A25234499,aminoglutethimide,glucocorticoid receptor antagonist
4,C-7161-01-LM6-001,A549,BRD-A25234499-001-18-3,B17,0.12346,2,48H,glucocorticoid receptor antagonist,CYP11A1|CYP19A1,-0.531468,...,-0.451350,-0.154301,0.866404,-0.086069,0.021753,-0.943961,1.091330,BRD-A25234499,aminoglutethimide,glucocorticoid receptor antagonist


In [9]:
# Load common compounds
common_file = pathlib.Path("..", "..", "..", "6.paper_figures", "data", "significant_compounds_by_threshold_both_assays.tsv.gz")
common_df = pd.read_csv(common_file, sep="\t")

common_compounds = common_df.compound.unique().tolist()
print(len(common_compounds))

1327


In [10]:
# Only calculate using common compounds
df_moa = df_all_moa.query("pert_iname in @common_compounds")

df_moa.shape

(8322, 1032)

In [11]:
# How many total MOAs in common
moa_list = (
    pd.DataFrame(
        pd.concat([
            pd.Series(x) for x in df_moa.moa.str.split("|")
        ])
        .dropna(), columns=['moa']
    )
)

moa_list.moa = moa_list.moa.str.lower()
moa_list = (
    pd.DataFrame(
        moa_list.moa.value_counts()
    )
    .reset_index()
    .rename(columns={"moa": "compound_count", "index": "moa"})
)

print(moa_list.moa.nunique())

511


In [12]:
# How many MOAs with greater than 3 compounds?
moa_list = moa_list.assign(num_unique_cpd=moa_list.compound_count / 6)
moa_list_subset = moa_list.query("num_unique_cpd > 3")

print(moa_list_subset.moa.nunique())

126


In [13]:
df_no_moa.shape

(78, 1032)

In [14]:
##list of "Broad samples" WITHOUT Mechanism of Actions (MOA) after aligning L1000 and Cell painting MOAs
df_no_moa['Metadata_broad_sample'].unique().tolist()

['BRD-K21547160-001-01-4',
 'BRD-A37288617-003-02-2',
 'BRD-A44448661-001-04-8',
 'BRD-A84045418-001-03-1',
 'BRD-K01192156-001-02-7',
 'BRD-K41438959-001-01-7',
 'BRD-K51033547-003-02-6',
 'BRD-K52080565-001-09-2',
 'BRD-K60623809-001-02-0',
 'BRD-K73395020-001-02-3',
 'BRD-A20131130-001-01-7',
 'BRD-K87278688-001-01-0',
 'BRD-M98279124-300-01-1']

### Next:

### - Get Correlation (using Spearman coefficient)  between compounds for all DOSES (1 - 6).

### - Then, Get the correlation values btw compounds of each particular MOA, and calculate the median from the correlation values.

## Recoding Dose Information

The Drug Repurposing Hub collected data on 6 to 7 dose points per compound.
In general, most doses are very near the following 7 dose points (mmoles per liter):

> [0.04, 0.12, 0.37, 1.11, 3.33, 10, 20]

Therefore, to make it easier to filter by dose when comparing compounds, we first align the doses collected in the dataset to their nearest dose point above.
We then recode the dose points into ascending numerical levels and add a new metadata annotation `dose_recode` to the consensus signatures.

| Dose | Dose Recode |
| :--: | :---------: |
| 0 (DMSO) | 0 |
| ~0.04 | 1 |
| ~0.12 | 2 |
| ~0.37 | 3 |
| ~1.11 | 4 |
| ~3.33 | 5 |
| ~10 | 6 |
| ~20 | 7 |

In [15]:
def get_median_score(moa_list, df_dose, df_cpd_agg):
    
    """
    Get the correlation values between compounds of each MOA, 
    then calculate the median of these correlation values 
    and assign it as the "median score" of the MOA.
    
    params: 
    moa_list: list of distinct moas for a particular dose
    df_dose: merged consensus and moa dataframe of a partcular dose
    df_dose_corr: merged consensus and moa dataframe of compound correlations of a particular dose

    Returns:
    moa_median_score: Dict with moa as the keys, and their median scores as the values
    moa_cpds: Dict with moa as the keys, and the list of moa for each moa as the values
    
    """
    
    moa_cpds = {}
    moa_median_score = {}
    for moa in moa_list:
        cpds = df_dose['pert_iname'][df_dose['moa'] == moa].unique().tolist()
        moa_cpds[moa] = cpds
        ##taking correlation btw cpds for each MOA
        df_cpds = df_cpd_agg.loc[cpds]
        cpds_corr = df_cpds.transpose().corr(method = 'spearman')
        if len(cpds) == 1:
            median_val = 1
        else:
            cpds_corr.index.name = "pert_iname_compare"
            cpds_corr = cpds_corr.reset_index().melt(id_vars="pert_iname_compare", value_name="spearman_corr")
            cpds_corr = cpds_corr.assign(keep_me_diff_comparison = cpds_corr.pert_iname_compare != cpds_corr.pert_iname)
            cpds_corr = cpds_corr.query("keep_me_diff_comparison")
            median_val = cpds_corr.spearman_corr.median()

        moa_median_score[moa] = median_val
        
    return moa_median_score, moa_cpds

In [16]:
def check_moa(moa_med_score, moa_cpds, df_moa):
    """
    Check if all distinct moas in the moa_consensus dataframe (df_moa) 
    are in moa_med_score & moa_cpd, if not add them as keys and give them
    a null value as the median score for moa_med_score and also as values for moa_cpds.
    
    params: 
    moa_med_score: Dict with moa as the keys, and their size as the values
    moa_cpds: Dict with moa as the keys, and the list of moa for each moa as the values
    data_moa: merged consensus and moa df with moas

    Returns:
    moa_med_score: Dict with moa as the keys, and their size as the values
    moa_cpds: Dict with moa as the keys, and the list of moa for each moa as the values
    
    """
    moa_list = df_moa['moa'].unique().tolist()
    moa_keys = moa_med_score.keys()
    for moa in moa_list:
        if moa not in moa_keys:
            moa_med_score[moa] = np.nan
            moa_cpds[moa] = np.nan
    return moa_med_score, moa_cpds

In [17]:
def get_moa_medianscores(df_moa):
    
    """
    Generate a dataframe of distinct moas with their median scores and
    corresponding list of compounds for different doses.
    
    params: 
    df_moa: merged consensus and moa dataframe

    Returns:
    df_moa_med_score: dataframe of distinct moas with their corresponding median scores 
    and list of compounds for all doses.
    
    """
    dose_list = list(set(df_moa['Metadata_dose_recode'].unique().tolist()))
    print(dose_list)
    for dose in dose_list:
        df_dose = df_moa[df_moa['Metadata_dose_recode'] == dose].copy()
        df_cpd_agg = df_dose.groupby(['pert_iname']).agg(['mean']).reset_index()
        df_cpd_agg.index = df_cpd_agg.pert_iname

        df_cpd_agg.drop(['pert_iname', 'Metadata_mmoles_per_liter', 'Metadata_dose_recode'], axis = 1, inplace = True)
        dose_moa_list = df_dose['moa'].unique().tolist()
        #get the median of the corr values of the cpds for each MOA
        dose_moa_med_score, dose_moa_cpds = get_median_score(dose_moa_list, df_dose, df_cpd_agg)
        #check if all moa in the df_moa is present in the dose_moa
        dose_moa_med_score, dose_moa_cpds = check_moa(dose_moa_med_score, dose_moa_cpds, df_moa)
        sorted_moa_med_score = {key:value for key, value in sorted(dose_moa_med_score.items(), key=lambda item: item[0])}
        sorted_dose_cpds = {key:value for key, value in sorted(dose_moa_cpds.items(), key=lambda item: item[0])}
        if dose == 1:
            df_moa_med_score = pd.DataFrame.from_dict(sorted_moa_med_score, orient='index', columns = ['dose_1'])
        else:
            df_moa_med_score['dose_' + str(dose)] = sorted_moa_med_score.values()
        df_moa_med_score['moa_cpds_dose_' + str(dose)] = list(sorted_dose_cpds.values())
            
    return df_moa_med_score

In [18]:
data_moa_med_score = get_moa_medianscores(df_moa)

print(data_moa_med_score.shape)
data_moa_med_score.head()

[1, 2, 3, 4, 5, 6, 7]


/Users/gway/miniconda3/envs/lincs-complimentarity/lib/python3.9/site-packages/pandas/core/generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
/Users/gway/miniconda3/envs/lincs-complimentarity/lib/python3.9/site-packages/pandas/core/generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
/Users/gway/miniconda3/envs/lincs-complimentarity/lib/python3.9/site-packages/pandas/core/generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)
/Users/gway/miniconda3/envs/lincs-complimentarity/lib/python3.9/site-packages/pandas/core/generic.py:4153: PerformanceWarning: dropping on a non-lexsorted

(583, 14)


/Users/gway/miniconda3/envs/lincs-complimentarity/lib/python3.9/site-packages/pandas/core/generic.py:4153: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


,dose_1,moa_cpds_dose_1,dose_2,moa_cpds_dose_2,dose_3,moa_cpds_dose_3,dose_4,moa_cpds_dose_4,dose_5,moa_cpds_dose_5,dose_6,moa_cpds_dose_6,dose_7,moa_cpds_dose_7
11-beta hydroxysteroid dehydrogenase inhibitor|acetylcholinesterase inhibitor|caspase inhibitor|hiv protease inhibitor|lipid peroxidase inhibitor|quorum sensing signaling modulator|steryl sulfatase inhibitor|tyrosine phosphatase inhibitor|atpase inhibitor|nfkb pathway inhibitor|stat inhibitor,1.0,[ursolic-acid],1.0,[ursolic-acid],1.0,[ursolic-acid],1.0,[ursolic-acid],1.0,[ursolic-acid],1.0,[ursolic-acid],NaN,NaN
11-beta-hsd1 inhibitor,1.0,[carbenoxolone],1.0,[carbenoxolone],1.0,[carbenoxolone],1.0,[carbenoxolone],1.0,[carbenoxolone],1.0,[carbenoxolone],NaN,NaN
"17,20 lyase inhibitor|cytochrome p450 inhibitor|steroid sulfatase inhibitor|androgen biosynthesis inhibitor",1.0,[abiraterone-acetate],1.0,[abiraterone-acetate],1.0,[abiraterone-acetate],1.0,[abiraterone-acetate],1.0,[abiraterone-acetate],1.0,[abiraterone-acetate],NaN,NaN
3-ketoacyl coa thiolase inhibitor,1.0,[trimetazidine],1.0,[trimetazidine],1.0,[trimetazidine],1.0,[trimetazidine],1.0,[trimetazidine],1.0,[trimetazidine],NaN,NaN
3beta-hydroxy-delta5-steroid dehydrogenase inhibitor,1.0,[trilostane],1.0,[trilostane],1.0,[trilostane],1.0,[trilostane],1.0,[trilostane],1.0,[trilostane],NaN,NaN


### - Exclude MOAs with median score 1 and only null values and  also columns with only null values

#### The reason why we are excluding MOAs with median value == 1, is because they have only ONE compound and as a result the medain correlation value will be just 1, and there will not be differences in values btw different doses.

In [19]:
def exclude_moa(df_moa_med_score):
    """
    Exclude MOAs with median score 1, with only null values, and also columns with only null values.
    
    params: 
    df_moa_med_score: dataframe of distinct moas with their corresponding median scores
    and list of compounds for all doses.

    Returns:
    df_moa_medians: dataframe of distinct moas with NO median values of 1 
    and their corresponding list of compounds for all doses.
    
    """
    moa_with_med_index = []
    for moa in df_moa_med_score.index.tolist():
        moa_values = df_moa_med_score.loc[moa]
        if all(y != 1.0 for y in moa_values):
            moa_with_med_index.append(moa)
    df_moa_medians = df_moa_med_score.loc[moa_with_med_index]
    null_columns = [col for col in df_moa_medians.columns 
                 if all(df_moa_medians[col].isnull())]
    null_moas = [moa for moa in df_moa_medians.index 
                 if all(df_moa_medians.loc[moa].isnull())]
    df_moa_medians.drop(null_columns, axis = 1, inplace = True)
    df_moa_medians.drop(null_moas, axis = 0, inplace = True)
    
    return df_moa_medians

In [20]:
data_moa_medians = exclude_moa(data_moa_med_score)

In [21]:
print(data_moa_medians.shape)
data_moa_medians.head()

(211, 12)


,dose_1,moa_cpds_dose_1,dose_2,moa_cpds_dose_2,dose_3,moa_cpds_dose_3,dose_4,moa_cpds_dose_4,dose_5,moa_cpds_dose_5,dose_6,moa_cpds_dose_6
5 alpha reductase inhibitor,0.011311,"[dutasteride, finasteride]",-0.022556,"[dutasteride, finasteride]",-0.061138,"[dutasteride, finasteride]",0.004437,"[dutasteride, finasteride]",-0.015314,"[dutasteride, finasteride]",0.140244,"[dutasteride, finasteride]"
acat inhibitor,0.004445,"[avasimibe, kd-023, ci-976]",0.023241,"[avasimibe, kd-023, ci-976]",-0.017093,"[avasimibe, kd-023, ci-976]",-0.080472,"[avasimibe, kd-023, ci-976]",-0.101490,"[avasimibe, kd-023, ci-976]",0.038194,"[avasimibe, kd-023, ci-976]"
acetylcholine receptor agonist,0.005142,"[abt-202, nicotine, diphenidol, cytisine, vare...",-0.008461,"[abt-202, nicotine, diphenidol, cytisine, vare...",0.008707,"[abt-202, nicotine, diphenidol, cytisine, vare...",0.000276,"[abt-202, nicotine, diphenidol, cytisine, vare...",0.005321,"[abt-202, nicotine, diphenidol, cytisine, vare...",0.017490,"[abt-202, nicotine, diphenidol, cytisine, vare..."
acetylcholine receptor antagonist,0.019262,"[atracurium, metixene, mecamylamine, solifenac...",0.013668,"[atracurium, metixene, mecamylamine, solifenac...",0.005836,"[atracurium, metixene, mecamylamine, solifenac...",0.004114,"[atracurium, metixene, mecamylamine, solifenac...",0.001829,"[atracurium, metixene, mecamylamine, solifenac...",0.026004,"[atracurium, metixene, mecamylamine, solifenac..."
acetylcholine release stimulant,0.009536,"[bifemelane, dmp-543]",-0.051150,"[bifemelane, dmp-543]",0.059360,"[bifemelane, dmp-543]",0.015360,"[bifemelane, dmp-543]",0.039470,"[bifemelane, dmp-543]",-0.036100,"[bifemelane, dmp-543]"


In [22]:
def seperate_cpds_values(df_moa_medians):
    """
    Seperate the list of compunds columns from the values columns in
    moa_median_dataframe
    
    params: 
    df_moa_medians: dataframe of distinct moas with NO median values of 1 
    and their corresponding list of compounds for all doses.

    Returns:
    df_moa_cpds: dataframe of distinct moas with only their corresponding 
    list of compounds for all doses.
    
    df_moa_values: dataframe of distinct moas with only their sizes for all doses.
    """
    dose_cols = [col for col in df_moa_medians.columns.tolist() 
                 if (col.startswith("dose_"))]
    df_moa_cpds = df_moa_medians.drop(dose_cols, axis = 1)
    df_moa_values = df_moa_medians.loc[:, dose_cols].copy()
    df_moa_values = df_moa_values.reset_index().rename(columns={"index": "moa"})
    df_moa_cpds = df_moa_cpds.reset_index().rename(columns={"index": "moa"})
    
    return df_moa_cpds, df_moa_values

In [23]:
data_moa_cpds, data_moa_values = seperate_cpds_values(data_moa_medians)
data_moa_cpds.head()

,moa,moa_cpds_dose_1,moa_cpds_dose_2,moa_cpds_dose_3,moa_cpds_dose_4,moa_cpds_dose_5,moa_cpds_dose_6
0,5 alpha reductase inhibitor,"[dutasteride, finasteride]","[dutasteride, finasteride]","[dutasteride, finasteride]","[dutasteride, finasteride]","[dutasteride, finasteride]","[dutasteride, finasteride]"
1,acat inhibitor,"[avasimibe, kd-023, ci-976]","[avasimibe, kd-023, ci-976]","[avasimibe, kd-023, ci-976]","[avasimibe, kd-023, ci-976]","[avasimibe, kd-023, ci-976]","[avasimibe, kd-023, ci-976]"
2,acetylcholine receptor agonist,"[abt-202, nicotine, diphenidol, cytisine, vare...","[abt-202, nicotine, diphenidol, cytisine, vare...","[abt-202, nicotine, diphenidol, cytisine, vare...","[abt-202, nicotine, diphenidol, cytisine, vare...","[abt-202, nicotine, diphenidol, cytisine, vare...","[abt-202, nicotine, diphenidol, cytisine, vare..."
3,acetylcholine receptor antagonist,"[atracurium, metixene, mecamylamine, solifenac...","[atracurium, metixene, mecamylamine, solifenac...","[atracurium, metixene, mecamylamine, solifenac...","[atracurium, metixene, mecamylamine, solifenac...","[atracurium, metixene, mecamylamine, solifenac...","[atracurium, metixene, mecamylamine, solifenac..."
4,acetylcholine release stimulant,"[bifemelane, dmp-543]","[bifemelane, dmp-543]","[bifemelane, dmp-543]","[bifemelane, dmp-543]","[bifemelane, dmp-543]","[bifemelane, dmp-543]"


In [24]:
data_moa_values.head(10)

,moa,dose_1,dose_2,dose_3,dose_4,dose_5,dose_6
0,5 alpha reductase inhibitor,0.011311,-0.022556,-0.061138,0.004437,-0.015314,0.140244
1,acat inhibitor,0.004445,0.023241,-0.017093,-0.080472,-0.101490,0.038194
2,acetylcholine receptor agonist,0.005142,-0.008461,0.008707,0.000276,0.005321,0.017490
3,acetylcholine receptor antagonist,0.019262,0.013668,0.005836,0.004114,0.001829,0.026004
4,acetylcholine release stimulant,0.009536,-0.051150,0.059360,0.015360,0.039470,-0.036100
5,acetylcholinesterase inhibitor,0.005717,0.016139,-0.005579,-0.027033,-0.023787,0.010609
6,adenosine receptor agonist,0.009584,-0.018228,0.004757,-0.014888,0.003473,0.010341
7,adenosine receptor antagonist,0.001704,0.010194,-0.010265,0.000895,-0.010807,0.041622
8,adrenergic inhibitor,0.005423,-0.028822,0.011740,0.049058,0.048479,0.077847
9,adrenergic receptor agonist,0.005016,0.002575,0.007999,0.003155,0.006044,0.008378


In [25]:
# Output analytical file
output_file = pathlib.Path("moa_sizes_consensus_datasets/cell_painting_moa_analytical_set_profiles.tsv.gz")
analytical_set_df = df_moa.query("moa in @data_moa_cpds.moa").query("Metadata_moa != 'unknown'").reset_index(drop=True)

print(analytical_set_df.shape)
analytical_set_df.to_csv(output_file, index=False, sep="\t")

(5574, 1032)


In [26]:
def get_moa_size(df_moa_cpds, df_moa_values):
    """
    This function computes the number of compunds in each MOA
    i.e. moa_size and returns dataframe including the moa_size column
    
    params:
    df_moa_cpds: dataframe of distinct moas with only their corresponding 
    list of compounds for all doses.
    
    df_moa_values: dataframe of distinct moas with only their median scores for all doses.
    
    Returns:
    df_moa_cpds: dataframe of distinct moas with only their corresponding 
    list of compounds for all doses including moa_size column.
    
    df_moa_values: dataframe of distinct moas with only their median scores 
    including moa_size column for all doses.
    """
    
    df_moa_cpd_copy = df_moa_cpds.set_index('moa').rename_axis(None, axis=0).copy()
    num_col = len(df_moa_cpd_copy.columns)
    
    moa_count = {}
    for moa in df_moa_cpd_copy.index:
        col_sum = 0
        for col in df_moa_cpd_copy.columns:
            col_sum += len(df_moa_cpd_copy.loc[moa, col])
        moa_count[moa] = round(col_sum/num_col)
    df_moa_cpds['moa_size'] = moa_count.values()
    df_moa_values['moa_size'] = moa_count.values()
    return df_moa_cpds, df_moa_values

In [27]:
data_moa_cpds, data_moa_values = get_moa_size(data_moa_cpds, data_moa_values)

### - Check if the MOAs have the same compounds in all the Doses

In [28]:
def check_moas_cpds_doses(df_moa_cpds):
    """
    check if moas have the same compounds in all doses,
    and return the moas that don't have the same numbers of compounds.
    
    params: 
    df_moa_cpds: dataframe of distinct moas with only their corresponding 
    list of compounds for all doses.

    Returns:
    df_moa_not_equals_cpds: dataframe of moas that don't have the same numbers of 
    compounds in all doses.
    
    """
    df_moa_cpds = df_moa_cpds.set_index('moa').rename_axis(None, axis=0).copy()
    df_moa_cpds.drop(['moa_size'], axis=1, inplace = True)
    moas_with_no_equal_cpds = [moa for moa in df_moa_cpds.index 
                               for num in range(len(df_moa_cpds.columns) - 1) 
                               if not ((df_moa_cpds.loc[moa, df_moa_cpds.columns[num]]) 
                                       == (df_moa_cpds.loc[moa, df_moa_cpds.columns[num+1]]))]
    df_moa_not_equals_cpds = df_moa_cpds.loc[set(moas_with_no_equal_cpds)]
    
    return df_moa_not_equals_cpds

In [29]:
data_moa_not_equals_cpds = check_moas_cpds_doses(data_moa_cpds) ##MOAs with not the same cpds in all doses

### - MOAS that do not have the same number of compounds in all Doses

In [30]:
for moa in data_moa_not_equals_cpds.index:
    print(moa)
    for idx, cols in enumerate(data_moa_not_equals_cpds.columns):
        print('Dose ' + str(idx+1) +':', data_moa_not_equals_cpds.loc[moa, cols])
    print('\n')

### - MOAS with their median scores for all doses

In [31]:
data_moa_values.head(10)

,moa,dose_1,dose_2,dose_3,dose_4,dose_5,dose_6,moa_size
0,5 alpha reductase inhibitor,0.011311,-0.022556,-0.061138,0.004437,-0.015314,0.140244,2
1,acat inhibitor,0.004445,0.023241,-0.017093,-0.080472,-0.101490,0.038194,3
2,acetylcholine receptor agonist,0.005142,-0.008461,0.008707,0.000276,0.005321,0.017490,8
3,acetylcholine receptor antagonist,0.019262,0.013668,0.005836,0.004114,0.001829,0.026004,21
4,acetylcholine release stimulant,0.009536,-0.051150,0.059360,0.015360,0.039470,-0.036100,2
5,acetylcholinesterase inhibitor,0.005717,0.016139,-0.005579,-0.027033,-0.023787,0.010609,2
6,adenosine receptor agonist,0.009584,-0.018228,0.004757,-0.014888,0.003473,0.010341,3
7,adenosine receptor antagonist,0.001704,0.010194,-0.010265,0.000895,-0.010807,0.041622,4
8,adrenergic inhibitor,0.005423,-0.028822,0.011740,0.049058,0.048479,0.077847,2
9,adrenergic receptor agonist,0.005016,0.002575,0.007999,0.003155,0.006044,0.008378,20


In [32]:
def conv_list_to_str_cols(df_moa_cpds):
    """This function convert columns values that are lists to strings"""
    
    moa_cpd_cols = [col for col in df_moa_cpds.columns.tolist() 
                 if (col.startswith("moa_cpds_"))]
    for col in moa_cpd_cols:
        df_moa_cpds[col] = df_moa_cpds[col].apply(lambda row: ';'.join(map(str, row)))
    return df_moa_cpds

In [33]:
def save_to_csv(df, path, file_name):
    """saves moa dataframes to csv"""
    
    if not os.path.exists(path):
        os.mkdir(path)
    
    df.to_csv(os.path.join(path, file_name), index=False)

In [34]:
save_to_csv(df_moa, 'moa_sizes_consensus_datasets', 'modz_consensus_data.csv')
save_to_csv(df_all_moa, 'moa_sizes_consensus_datasets', 'modz_consensus_data_all_compounds.csv')

In [35]:
save_to_csv(conv_list_to_str_cols(data_moa_cpds), 'moa_sizes_consensus_datasets', 'cellpainting_moa_compounds.csv')

In [36]:
save_to_csv(data_moa_values, 'moa_sizes_consensus_datasets', 'modz_moa_median_scores.csv')

In [37]:
# Output files for visualization
cpd_summary_file = pathlib.Path("moa_sizes_consensus_datasets/matching_score_per_MOA_CellPainting.tsv.gz")

dose_recode_info = {
    'dose_1': '0.04 uM', 'dose_2':'0.12 uM', 'dose_3':'0.37 uM',
    'dose_4': '1.11 uM', 'dose_5':'3.33 uM', 'dose_6':'10 uM'
}

cpd_score_summary_df = (
    data_moa_values
    .rename(columns={"moa_size": "no_of_replicates"})
    .melt(
        id_vars=["moa", "no_of_replicates"],
        value_vars=["dose_1", "dose_2", "dose_3", "dose_4", "dose_5", "dose_6"],
        var_name="dose",
        value_name="matching_score"
    )
)

cpd_score_summary_df.dose = cpd_score_summary_df.dose.replace(dose_recode_info)

cpd_score_summary_df.to_csv(cpd_summary_file, sep="\t", index=False)
cpd_score_summary_df.head()

,moa,no_of_replicates,dose,matching_score
0,5 alpha reductase inhibitor,2,0.04 uM,0.011311
1,acat inhibitor,3,0.04 uM,0.004445
2,acetylcholine receptor agonist,8,0.04 uM,0.005142
3,acetylcholine receptor antagonist,21,0.04 uM,0.019262
4,acetylcholine release stimulant,2,0.04 uM,0.009536
